진호님이 일단 실행되게끔 만들어 본것

In [1]:
# 3_21 모델학습 후 예측한 결과가 심통치않아 새로 전처리 해보기로 하였다.
# 벡터를 이용한 모델을 생성해서 문자열 값을 리턴 받아보자.
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
import tensorflow as tf
import numpy as np

#1. Naver Sentiment Movie Corpus v1.0 다운로드
train_file = tf.keras.utils.get_file('train.txt', 'https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt')
test_file = tf.keras.utils.get_file( 'test.txt',  'https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt')

#파일을 이진모드로 읽어온다. 디코드는 utf8로한다.
train_text = open(train_file,'rb').read().decode(encoding='utf-8')
test_text = open(test_file,'rb').read().decode(encoding='utf-8')



In [38]:
import pandas as pd
#x,y값을 추출후 데이터프레임에 넣는다.
# X 값 추출 피쳐feature, y값 추출target 

df_train = pd.DataFrame({
    'feature':[row.split('\t')[1] for row in train_text.split('\n')[1:] if row.count('\t')>0],
    'target': [ int(row.split('\t')[2]) for row in train_text.split('\n')[1:] if row.count('\t')>0]
})


In [39]:
df_train
# print(y_train[:3])

,feature,target
0,아 더빙.. 진짜 짜증나네요 목소리,0
1,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,너무재밓었다그래서보는것을추천한다,0
3,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
...,...,...
149995,인간이 문제지.. 소는 뭔죄인가..,0
149996,평점이 너무 낮아서...,1
149997,이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?,0
149998,청춘 영화의 최고봉.방황과 우울했던 날들의 자화상,1


In [23]:
df.iloc[:,1]==0

0          True
1         False
2          True
3          True
4         False
          ...  
149995     True
149996    False
149997     True
149998    False
149999     True
Name: 0, Length: 150000, dtype: bool

In [ ]:


bad = df[df.iloc[:,1]==0]
good = df[df.iloc[:,1]==1]

test111=pd.concat([bad,good],axis=0) #라벨이 같이 들어온다.


from sklearn.feature_extraction.text import TfidfVectorizer
from konlpy.tag import Okt

vectorizer = TfidfVectorizer( tokenizer= Okt().morphs ) # 매개변수를 조정하여 다양한 옵션을 적용할 수 있습니다.
X = vectorizer.fit_transform(test111.iloc[:,0])
y = test111.iloc[:,1]

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
import numpy as np

X_train_dense = np.expand_dims( X_train.toarray(), axis=-1 )
X_test_dense = np.expand_dims( X_test.toarray(), axis=-1 )

In [8]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Input

# 모델 구성
model = Sequential([
    Input(shape=(X_train_dense.shape[1],)),
    Dense(128, activation='relu'),
    Dense(1, activation='sigmoid')
])

# 모델 컴파일 및 학습
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train_dense.squeeze(), y_train, epochs = 10, batch_size = 32, validation_split = 0.2)

c:\Users\uk246\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\data_adapter.py:1696: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  return t[start:end]


Epoch 1/10
200/200 [==============================] - 12s 51ms/step - loss: 0.5811 - accuracy: 0.7534 - val_loss: 0.4435 - val_accuracy: 0.8062
Epoch 2/10
200/200 [==============================] - 11s 55ms/step - loss: 0.2700 - accuracy: 0.9167 - val_loss: 0.3958 - val_accuracy: 0.8194
Epoch 3/10
200/200 [==============================] - 10s 50ms/step - loss: 0.1188 - accuracy: 0.9742 - val_loss: 0.4058 - val_accuracy: 0.8175
Epoch 4/10
200/200 [==============================] - 10s 48ms/step - loss: 0.0599 - accuracy: 0.9898 - val_loss: 0.4296 - val_accuracy: 0.8181
Epoch 5/10
200/200 [==============================] - 10s 48ms/step - loss: 0.0346 - accuracy: 0.9958 - val_loss: 0.4601 - val_accuracy: 0.8131
Epoch 6/10
200/200 [==============================] - 10s 52ms/step - loss: 0.0225 - accuracy: 0.9970 - val_loss: 0.4826 - val_accuracy: 0.8094
Epoch 7/10
200/200 [==============================] - 10s 49ms/step - loss: 0.0159 - accuracy: 0.9980 - val_loss: 0.5040 - val_accuracy:

In [4]:
# !pip install konlpy


In [20]:
test = ["비가 오니까 약속을 취소해야 되는게 너무 싫다."]

# 학습된 TF-IDF 벡터라이저로 테스트 데이터 변환
X_test_sample = vectorizer.transform(test)
X_test_sample_dense = X_test_sample.toarray()

# 모델을 사용하여 예측 수행
predictions = model.predict(X_test_sample_dense)

# 예측 결과 출력 (긍정: 1, 부정: 0)
print("예측 결과:", np.round(predictions).astype(int))

1/1 [==============================] - 0s 74ms/step
예측 결과: [[0]]


In [ ]:
# model.save('기본 DNN 적용모델.h5')